## Using geographical data from movies
ODMax searches for geographical data in user-provided videos. If these are found, then outcoming stills will be provided with latitude, longitude and elevation coordinates. This notebook demonstrates this workflow and demonstrates how you can plot the data to get an idea of what coverage you have over your 360-degree video extracts.

### Note
This notebook requires `exiftool` to be installed. Please refer to the installation instructions on https://odmax.readthedocs.io if you do not yet have `exiftool` installed on your system.

### Import packages
let's first import the necessary packages for this notebook. We also make one convenience function to extract coordinates from JPG files later one

In [ ]:
%matplotlib inline
import os
import odmax
import matplotlib.pyplot as plt
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs

def get_exif(fn):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    image = Image.open(fn)
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]
                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value
    return exif_data


We need a large enough file to work with. Below we download a file containing 50 FPS data from a GoPro 360 camera platform.

In [ ]:
!wget https://object-store.rc.nectar.org.au/v1/AUTH_9f7c80bfd20f45bebc780b06c405f0df/asdc-public/GOPR0011_1599383304667.mp4

Let us check if the file is indeed present in the current folder

In [ ]:
ls

First, we will use the API to open the file as a `odmax.Video` object. If geographical information is found, this will be indicated and the first valid and complete point available with a coordinate and time stamp will be displayed with it.

In [ ]:
import odmax
video_file = "GOPR0011_1599383304667.mp4"
Video = odmax.Video(video_file)


When `exiftool` is properly installed you should see a first location and time stamp displayed above. The actual gps information is stored in a property called `gdf_gps`

In [ ]:
Video.gdf_gps

This is a Geopandas DataFrame (i.e. gdf) which holds a geometry. We have a convenience method to plot this

In [ ]:
Video.plot_gps()

Let's look a bit closer at the plotting options

In [ ]:
help(Video.plot_gps)

We can use `cartopy` to further improve the plots and add background WMTS services. Let's try that with OpenStreetMap at a zoom level of 18 (make sure you install `cartopy` with `conda install cartopy`. `pip install` is very difficult).

In [ ]:
Video.plot_gps(
    geographical=True,
    tiles="OSM",
    zoom_level=18,
    plot_kwargs={"color": "r", "marker": "x"}
)



Another option is to use a satellite background by choosing a different WMTS service.

In [ ]:
Video.plot_gps(
    geographical=True,
    tiles="QuadtreeTiles",
    zoom_level=18,
    plot_kwargs={"color": "r", "marker": "x"}
)



Now let's extract 20 Frames from this track and store the latitudes and longitudes in two arrays. We'll also store the stills in .jpg files.

In [ ]:
frames = list(range(1150, 1650, 25))
lons, lats, fns = ([], [], [])
path = "geotest"
# ensure the path exists
if not(os.path.isdir(path)):
    os.makedirs(path)
    
for f in frames:
    print(f"Extracting frame {f}")
    Frame = Video.get_frame(f)
    # File naming will be automated based on path and prefix. Default prefix is "still"
    fn = Frame.to_file(path)
    # keep track of the files
    fns.append(fn)
    # also store the lat and lon coordinate
    lons.append(Frame.coord.lon)
    lats.append(Frame.coord.lat)
    


We will also read the files back in memory and use a few PIL functions to extract the latitude and longitude coordinates from the .jpg. In this way we can make sure that the locations are written in the .jpgs properly.

In [ ]:
lons2, lats2 = ([], [])
for fn in fns:
    exif = get_exif(fn)
    l = exif["GPSInfo"]["GPSLatitude"]
    lat = l[0] + l[1]/60. + l[2]/3600
    if exif["GPSInfo"]["GPSLatitudeRef"] == "S":
        lat *= -1
    l = exif["GPSInfo"]["GPSLongitude"]
    lon = l[0] + l[1]/60. + l[2]/3600
    if exif["GPSInfo"]["GPSLongitudeRef"] == "W":
        lon *= -1
    lons2.append(lon)
    lats2.append(lat)


Now plot all the information together to see if odmax accurately managed to geotag the stills.

In [ ]:
ax = Video.plot_gps(geographical=True, figsize=(16, 10), tiles="OSM", crs=cimgt.GoogleTiles().crs, zoom_level=18, plot_kwargs={"color": "k", "marker": ".", "label": "original video"})
ax.plot(lons, lats, "o", markersize=12, transform=ccrs.PlateCarree(), zorder=2, label="before file writing")
ax.plot(lons2, lats2, "x", markersize=10, color="r", transform=ccrs.PlateCarree(), zorder=3, label="after file writing")
plt.legend()

# retrieve the current counding box in geographical coordinates
bbox = list(ax.get_extent(crs=ccrs.PlateCarree()))
print(bbox)
# make the box a bit more zoomed in
bbox[2] +=0.7*(bbox[3]-bbox[2])
ax.set_extent(bbox, crs=ccrs.PlateCarree())

